# **Data Preprocessing**

In this notebook we aim to prepare the data for modelling.
Data preprocessing will take place in 6 stages:

* **1. Loading in the Data**
* **2. Renaming the Dataframe Fields**
* **3. Merging the Dataframes**
* **4. Missing Values**
* **5. Creating New Variables**
* **6. Adding Dummy Variables**

After preprocessing is complete, the data will be ready for training a machine learning model 

## **Dependencies**

In [28]:
import pandas as pd
import numpy as np

from dateutil.relativedelta import *

## **1. Loading in the Data**

We are going to read in our six dataframes directly from Github, which will prevent other users from having to locally download the csv files

In [3]:
race_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/race.csv'
races = pd.read_csv(race_url)

results_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/results.csv'
results = pd.read_csv(results_url)

qualifying_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/qualifying_results.csv'
qualifying = pd.read_csv(qualifying_url)

driver_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/driver_standings.csv'
driver_standings = pd.read_csv(driver_url)

constructor_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/constructor_standings.csv'
constructor_standings = pd.read_csv(constructor_url)

weather_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/weather_info.csv'
weather = pd.read_csv(weather_url)

## **2. Renaming the Dataframe Fields**

Before we can merge the dataframes, we need to rename any fields that have differing names:

In [14]:
qualifying.rename(columns = {'grid_position': 'grid'}, inplace = True)

print(qualifying.shape)
qualifying.head()

(14559, 6)


,grid,driver_name,car,qualifying_time,season,round
0,1,Keke Rosberg ROS,Williams Honda,1:34.526,1983,1
1,2,Alain Prost PRO,Renault,1:34.672,1983,1
2,3,Patrick Tambay TAM,Ferrari,1:34.758,1983,1
3,4,Nelson Piquet PIQ,Brabham BMW,1:35.114,1983,1
4,5,Derek Warwick WAR,Toleman Hart,1:35.206,1983,1


## **3. Merging the Dataframes**

We can now begin merging the dataframes. Since there is no common identifier (which is one-to-one) between the dataframes, we will iteratively add two dataframes together which have a common key. This will be done until all six dataframes are contained within a single dataframe:

In [26]:
df1 = pd.merge(races, weather, how='inner', on=['season', 'round', 'circuit_id']).drop(['lat', 'long','country','weather'], axis = 1)
df2 = pd.merge(df1, results, how='inner', on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time'], axis = 1)

df3 = pd.merge(df2, driver_standings, how='left', on=['season', 'round', 'driver']) 
df4 = pd.merge(df3, constructor_standings, how='left', on=['season', 'round', 'constructor']) # from 1958

final_df = pd.merge(df4, qualifying, how='inner', on=['season', 'round', 'grid']).drop(['driver_name', 'car'], axis = 1) # from 1983

In [27]:
print(final_df.shape)
final_df.head()

(14536, 22)


,season,round,circuit_id,date,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,...,constructor,grid,podium,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qualifying_time
0,1983,1,jacarepagua,1983-03-13,0,0,1,0,0,piquet,...,brabham,4,1,0.0,0.0,0.0,0.0,0.0,0.0,1:35.114
1,1983,1,jacarepagua,1983-03-13,0,0,1,0,0,lauda,...,mclaren,9,2,0.0,0.0,0.0,0.0,0.0,0.0,1:36.054
2,1983,1,jacarepagua,1983-03-13,0,0,1,0,0,laffite,...,williams,18,3,0.0,0.0,0.0,0.0,0.0,0.0,1:38.234
3,1983,1,jacarepagua,1983-03-13,0,0,1,0,0,tambay,...,ferrari,3,4,0.0,0.0,0.0,0.0,0.0,0.0,1:34.758
4,1983,1,jacarepagua,1983-03-13,0,0,1,0,0,surer,...,arrows,20,5,0.0,0.0,0.0,0.0,0.0,0.0,1:38.468


## **4. Missing Values**

Before we can begin to deal with missing values, we need to identify which fields contain missing values:

In [29]:
final_df.isna().sum()

season                         0
round                          0
circuit_id                     0
date                           0
weather_warm                   0
weather_cold                   0
weather_dry                    0
weather_wet                    0
weather_cloudy                 0
driver                         0
date_of_birth                  0
nationality                    0
constructor                    0
grid                           0
podium                         0
driver_points                563
driver_wins                  563
driver_standings_pos         563
constructor_points           164
constructor_wins             164
constructor_standings_pos    164
qualifying_time              260
dtype: int64

As can be seen, it is only the final 7 fields that contain any missing data. For the fields with the most missing data (driver_points, driver_wins & driver_standing_pos), this is approximately only 3.9% of the data available. Therefore, any missing value imputation or removal should not have a significant effect on the final performance of the model 

In [31]:
# fill/drop nulls

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins' , 'constructor_standings_pos']:
    final_df[col].fillna(0, inplace = True)
    final_df[col] = final_df[col].map(lambda x: int(x))
    
final_df.dropna(inplace = True )

## **5. Creating New Variables**

At this point, our dataframe contains information that can be altered to create new variables that may offer greater utility.

For example, instead of including the driver's data of birth as a variables, let's instead work out the age of the driver at the time of a given Grand Prix:

In [41]:
final_df['date'] = pd.to_datetime(final_df.date)
final_df['date_of_birth'] = pd.to_datetime(final_df.date_of_birth)
final_df['driver_age'] = final_df.apply(lambda x: relativedelta(x['date'], x['date_of_birth']).years, axis=1)
final_df.drop(['date', 'date_of_birth'], axis = 1, inplace = True)

AttributeError: 'DataFrame' object has no attribute 'date'

In [42]:
final_df

,season,round,circuit_id,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,nationality,...,grid,podium,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qualifying_time,driver_age
14,1983,1,jacarepagua,0,0,1,0,0,keke_rosberg,Finnish,...,1,15,0,0,0,0,0,0,0.000,34
5,1983,1,jacarepagua,0,0,1,0,0,prost,French,...,2,6,0,0,0,0,0,0,0.146,28
3,1983,1,jacarepagua,0,0,1,0,0,tambay,French,...,3,4,0,0,0,0,0,0,0.232,33
0,1983,1,jacarepagua,0,0,1,0,0,piquet,Brazilian,...,4,1,0,0,0,0,0,0,0.588,30
6,1983,1,jacarepagua,0,0,1,0,0,warwick,British,...,5,7,0,0,0,0,0,0,0.680,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14531,2019,21,yas_marina,1,0,0,0,0,giovinazzi,Italian,...,16,16,14,0,17,57,0,8,3.335,25
14528,2019,21,yas_marina,1,0,0,0,0,raikkonen,Finnish,...,17,13,43,0,12,57,0,8,3.604,40
14532,2019,21,yas_marina,1,0,0,0,0,russell,British,...,18,17,0,0,20,1,0,10,3.938,21
14534,2019,21,yas_marina,1,0,0,0,0,kubica,Polish,...,19,19,1,0,19,1,0,10,4.457,34


Instead of now having one variable for the data of birth of the driver and another for the date of the Grand Prix, we can collapse those two variables into a single variables which is the driver's age at the time of the race

We can perform a similar idea when it comes to the qualifying time of the drivers. 

In [43]:
final_df['qualifying_time'] = final_df.qualifying_time.map(lambda x: 0 if str(x) == '00.000' 
                             else(float(str(x).split(':')[1]) + (60 * float(str(x).split(':')[0])) if x != 0 else 0))
final_df = final_df[final_df['qualifying_time'] != 0]
final_df.sort_values(['season', 'round', 'grid'], inplace = True)
final_df['qualifying_time_diff'] = final_df.groupby(['season', 'round']).qualifying_time.diff()
final_df['qualifying_time'] = final_df.groupby(['season', 'round']).qualifying_time_diff.cumsum().fillna(0)
final_df.drop('qualifying_time_diff', axis = 1, inplace = True)

IndexError: list index out of range

## **6. Adding Dummy Variables**

Categorical variables like the circuit, constructor and nationality are potentially important variables that should be included in this analysis. This means that they need to be recoded as dummy variables. However, the problem with this is that if we include every nationality, circuit and constructor, the dimensionality of our final dataset will be too large. To prevent this, I have created some conditions to remove circuits, nationalities and constructors that have not partaken in a 'enough' races:

In [44]:
# get dummies

df_dum = pd.get_dummies(final_df, columns = ['circuit_id', 'nationality', 'constructor'] )

for col in df_dum.columns:
    if 'nationality' in col and df_dum[col].sum() < 140:
        df_dum.drop(col, axis = 1, inplace = True)
        
    elif 'constructor' in col and df_dum[col].sum() < 140:
        df_dum.drop(col, axis = 1, inplace = True)
        
    elif 'circuit_id' in col and df_dum[col].sum() < 70:
        df_dum.drop(col, axis = 1, inplace = True)
    
    else:
        pass

We have now obtained our final, complete dataset. As is evident, this dataset contains 100 features, which may lead to overfitting when it comes to training the machine learning model. If this proves to be the case, I will conduct PCA and other techniques to reduce the dimensions

In [46]:
print(df_dum.shape)
df_dum

(14272, 100)


,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,podium,...,constructor_minardi,constructor_prost,constructor_red_bull,constructor_renault,constructor_sauber,constructor_team_lotus,constructor_toro_rosso,constructor_toyota,constructor_tyrrell,constructor_williams
14,1983,1,0,0,1,0,0,keke_rosberg,1,15,...,0,0,0,0,0,0,0,0,0,1
5,1983,1,0,0,1,0,0,prost,2,6,...,0,0,0,1,0,0,0,0,0,0
3,1983,1,0,0,1,0,0,tambay,3,4,...,0,0,0,0,0,0,0,0,0,0
0,1983,1,0,0,1,0,0,piquet,4,1,...,0,0,0,0,0,0,0,0,0,0
6,1983,1,0,0,1,0,0,warwick,5,7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14531,2019,21,1,0,0,0,0,giovinazzi,16,16,...,0,0,0,0,0,0,0,0,0,0
14528,2019,21,1,0,0,0,0,raikkonen,17,13,...,0,0,0,0,0,0,0,0,0,0
14532,2019,21,1,0,0,0,0,russell,18,17,...,0,0,0,0,0,0,0,0,0,1
14534,2019,21,1,0,0,0,0,kubica,19,19,...,0,0,0,0,0,0,0,0,0,1


In [47]:
# here is a list of all the features:

list(df_dum.columns.values)

['season',
 'round',
 'weather_warm',
 'weather_cold',
 'weather_dry',
 'weather_wet',
 'weather_cloudy',
 'driver',
 'grid',
 'podium',
 'driver_points',
 'driver_wins',
 'driver_standings_pos',
 'constructor_points',
 'constructor_wins',
 'constructor_standings_pos',
 'qualifying_time',
 'driver_age',
 'circuit_id_BAK',
 'circuit_id_adelaide',
 'circuit_id_albert_park',
 'circuit_id_americas',
 'circuit_id_bahrain',
 'circuit_id_brands_hatch',
 'circuit_id_catalunya',
 'circuit_id_detroit',
 'circuit_id_estoril',
 'circuit_id_galvez',
 'circuit_id_hockenheimring',
 'circuit_id_hungaroring',
 'circuit_id_imola',
 'circuit_id_indianapolis',
 'circuit_id_interlagos',
 'circuit_id_istanbul',
 'circuit_id_jacarepagua',
 'circuit_id_jerez',
 'circuit_id_kyalami',
 'circuit_id_magny_cours',
 'circuit_id_marina_bay',
 'circuit_id_monaco',
 'circuit_id_monza',
 'circuit_id_nurburgring',
 'circuit_id_osterreichring',
 'circuit_id_phoenix',
 'circuit_id_red_bull_ring',
 'circuit_id_ricard',
 'c